In [12]:
# load dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [13]:
# Load data and select trainging hour and test hour
trainhour = 5
testhour = 7
df = pd.read_csv('series.csv')
cols = ['21', '22', '23', '24', '25', '26', '27', '28', '29', '30']
df['sum'] = df[cols].sum(axis=1)
data = df.drop(df[(df['hour'] != trainhour)].index)
testdata = df.drop(df[(df['hour'] != testhour)].index)
del df

In [14]:
# Categorize the 'sum' column the number of PIPs over selected range
def setlabels(x):
    if x < -10:
        return -1
    elif x > 10:
        return 1
    else:
        return 0

data["labels"] = data["sum"].apply(setlabels)

In [15]:
# Get X, y for model
X = data.iloc[:,8:28].to_numpy()

from sklearn.preprocessing import LabelEncoder
y_encode = data['labels']
label_encoder = LabelEncoder()
label_encoder.fit(y_encode)
y = label_encoder.transform(y_encode)
y[0:10]

array([1, 1, 1, 2, 2, 2, 2, 1, 1, 1], dtype=int64)

In [16]:
# Split data into training and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [17]:
# Create, fit, score model
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
print(f"Model Result (train): {clf.score(X_train, y_train)}")
print(f"Model Result (test): {clf.score(X_test, y_test)}")

Model Result (train): 1.0
Model Result (test): 0.8962409886714727


In [18]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
print(f"Model Result (train): {rf.score(X_train, y_train)}")
print(f"Model Result (test): {rf.score(X_test, y_test)}")

Model Result (train): 1.0
Model Result (test): 0.9541709577754892


In [19]:
# Pull out validation data and make predicts
formodel = testdata.iloc[:,8:28].to_numpy()
testdata['class'] = label_encoder.inverse_transform(rf.predict(formodel))
testdata = testdata.drop(testdata[(testdata['class'] == 0)].index)
testdata.shape

(6, 70)

In [20]:
totalrows = testdata.shape[0]
totalcols = testdata.shape[1]
profit = []
for i in range(totalrows):
    temp = 0
    for j in range(10):
        temp += testdata.iloc[i,28+j]
        if (testdata.iloc[i,totalcols-1] == -1):
            if(temp > 2):
              break  
        else:
            if(temp < -2):
              break
    
    if (testdata.iloc[i,totalcols-1] == -1):
        temp = -temp
    profit.append(temp)

print("Est Profit: " + str(sum(profit)))

Est Profit: 4.799999999999999


In [21]:
#testdata.to_csv('binarytest.csv')